In [ ]:
# ============================================================================
# PHASE 1A: CPU SERIAL BASELINE
# ============================================================================

import pandas as pd
import numpy as np
import time
import json
import os
import psutil
from datetime import datetime
import gc

print("="*80)
print("PHASE 1A: CPU SERIAL BASELINE")
print("NYC REAL ESTATE PROJECT")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Working directory: {os.getcwd()}")

# Verify session resources
print(f"\nSession Resources:")
print(f"   CPU cores available: {psutil.cpu_count()}")
print(f"   Total memory: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"   Available memory: {psutil.virtual_memory().available / (1024**3):.1f} GB")

# Verify dataset
data_path = 'nyc-property-sales.csv'
if os.path.exists(data_path):
    size_mb = os.path.getsize(data_path) / (1024**2)
    print(f"\n✓ Dataset found: {data_path}")
    print(f"✓ Dataset size: {size_mb:.2f} MB")
else:
    print(f"\n✗ ERROR: Dataset not found!")
    raise FileNotFoundError(f"Dataset not found at {data_path}")

# Force single-threaded execution
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

print("\n✓ Environment set to single-threaded mode")
print("\nStarting Phase 1A - CPU Serial Baseline...")

# Initialize timing dictionary
timings = {}

# START PHASE 1A TIMER
phase1a_total_start = time.time()

# STEP 1: DATA LOADING
print("\n" + "="*80)
print("STEP 1: DATA LOADING")
print("="*80)
step_start = time.time()
df_1a = pd.read_csv('nyc-property-sales.csv')
timings['data_loading'] = time.time() - step_start

# CLEAN COLUMN NAMES (FIX #1)
df_1a.columns = df_1a.columns.str.strip()

print(f"✓ Loaded {len(df_1a):,} rows and {len(df_1a.columns)} columns")
print(f"✓ Memory usage: {df_1a.memory_usage(deep=True).sum() / (1024**3):.2f} GB")
print(f"  Time: {timings['data_loading']:.2f}s")

# STEP 2: FILTERING
print("\n" + "="*80)
print("STEP 2: PREPROCESSING (FILTERING)")
print("="*80)
step_start = time.time()
initial_rows = len(df_1a)
print(f"Initial rows: {initial_rows:,}")

# Convert SALE PRICE to numeric
print("\nConverting SALE PRICE to numeric...")
df_1a['SALE PRICE'] = pd.to_numeric(df_1a['SALE PRICE'], errors='coerce')
nan_prices = df_1a['SALE PRICE'].isna().sum()
if nan_prices > 0:
    print(f"  Found {nan_prices:,} invalid SALE PRICE values -> removing")
    df_1a = df_1a.dropna(subset=['SALE PRICE']).copy()

# Remove zero-dollar sales
df_1a = df_1a[df_1a['SALE PRICE'] > 0].copy()
print(f"After removing $0 sales: {len(df_1a):,} rows")

# Filter price range ($100K - $20M)
df_1a = df_1a[(df_1a['SALE PRICE'] >= 100000) & (df_1a['SALE PRICE'] <= 20000000)].copy()
print(f"After price filtering ($100K-$20M): {len(df_1a):,} rows")

timings['filtering'] = time.time() - step_start
rows_removed = initial_rows - len(df_1a)
print(f"\nRows removed: {rows_removed:,} ({rows_removed/initial_rows*100:.2f}%)")
print(f"Rows remaining: {len(df_1a):,} ({len(df_1a)/initial_rows*100:.2f}%)")
print(f"  Time: {timings['filtering']:.2f}s")

# STEP 3: MISSING VALUES
print("\n" + "="*80)
print("STEP 3: HANDLE MISSING VALUES")
print("="*80)
step_start = time.time()

# CONVERT TO NUMERIC FIRST (before computing median)
df_1a['LAND SQUARE FEET'] = pd.to_numeric(df_1a['LAND SQUARE FEET'], errors='coerce')
df_1a['GROSS SQUARE FEET'] = pd.to_numeric(df_1a['GROSS SQUARE FEET'], errors='coerce')
df_1a['YEAR BUILT'] = pd.to_numeric(df_1a['YEAR BUILT'], errors='coerce')

# LAND SQUARE FEET
missing_land = df_1a['LAND SQUARE FEET'].isnull().sum()
if missing_land > 0:
    median_land = df_1a['LAND SQUARE FEET'].median()
    df_1a['LAND SQUARE FEET'].fillna(median_land, inplace=True)
    print(f"✓ LAND SQUARE FEET: {missing_land:,} missing -> filled with median {median_land:,.0f}")

# GROSS SQUARE FEET
missing_gross = df_1a['GROSS SQUARE FEET'].isnull().sum()
if missing_gross > 0:
    median_gross = df_1a['GROSS SQUARE FEET'].median()
    df_1a['GROSS SQUARE FEET'].fillna(median_gross, inplace=True)
    print(f"✓ GROSS SQUARE FEET: {missing_gross:,} missing -> filled with median {median_gross:,.0f}")

# YEAR BUILT
missing_year = df_1a['YEAR BUILT'].isnull().sum()
if missing_year > 0:
    median_year = df_1a['YEAR BUILT'].median()
    df_1a['YEAR BUILT'].fillna(median_year, inplace=True)
    print(f"✓ YEAR BUILT: {missing_year:,} missing -> filled with median {median_year:.0f}")

timings['missing_values'] = time.time() - step_start
print(f"  Time: {timings['missing_values']:.2f}s")

# STEP 4: DATE CONVERSION
print("\n" + "="*80)
print("STEP 4: DATE CONVERSION")
print("="*80)
step_start = time.time()
df_1a['SALE DATE'] = pd.to_datetime(df_1a['SALE DATE'], errors='coerce')
null_dates = df_1a['SALE DATE'].isnull().sum()
print(f"✓ Converted SALE DATE to datetime")
if null_dates > 0:
    print(f"    Warning: {null_dates:,} dates failed conversion")
else:
    print(f"  ✓ All dates converted successfully")
    print(f"  Date range: {df_1a['SALE DATE'].min()} to {df_1a['SALE DATE'].max()}")
timings['date_conversion'] = time.time() - step_start
print(f"  Time: {timings['date_conversion']:.2f}s")

# STEP 5: SAVE DATA
print("\n" + "="*80)
print("STEP 5: SAVE PREPROCESSED DATA")
print("="*80)
output_path = 'data/processed/preprocessed_cpu_serial.csv'
os.makedirs('data/processed', exist_ok=True)
step_start = time.time()
df_1a.to_csv(output_path, index=False)
timings['save_data'] = time.time() - step_start
output_size = os.path.getsize(output_path) / (1024**2)
print(f"✓ Saved to: {output_path}")
print(f"✓ Output size: {output_size:.2f} MB")
print(f"  Time: {timings['save_data']:.2f}s")

# END PHASE 1A TIMER
phase1a_total = time.time() - phase1a_total_start
timings['total'] = phase1a_total

# FINAL SUMMARY
print("\n" + "="*80)
print("PHASE 1A COMPLETE!")
print("="*80)
print(f"Final dataset: {len(df_1a):,} rows, {len(df_1a.columns)} columns")
print(f"Memory usage: {df_1a.memory_usage(deep=True).sum() / (1024**3):.2f} GB")
print(f"\nTOTAL PHASE 1A TIME: {phase1a_total:.2f}s ({phase1a_total/60:.2f} minutes)")

# Timing breakdown
print("\nPhase 1A Timing Breakdown:")
for op, t in timings.items():
    if op != 'total':
        print(f"  • {op:20s}: {t:8.2f}s")
print(f"  {'─'*35}")
print(f"  • {'TOTAL':20s}: {phase1a_total:8.2f}s")

# Save timings (FIX #2 - cleaner structure)
timings_output = {
    'phase': 'phase1a_cpu_serial',
    'timestamp': datetime.now().isoformat(),
    'timings': timings,
    'dataset_info': {
        'final_rows': int(len(df_1a)),
        'final_columns': int(len(df_1a.columns)),
        'memory_gb': float(df_1a.memory_usage(deep=True).sum() / (1024**3)),
        'output_size_mb': float(output_size)
    }
}

os.makedirs('results/timings', exist_ok=True)
with open('results/timings/phase1a_timings.json', 'w') as f:
    json.dump(timings_output, f, indent=2)
print(f"\n✓ Timings saved to: results/timings/phase1a_timings.json")

# Clear memory
print("\n🧹 Clearing memory...")
del df_1a
gc.collect()
print("✓ Memory cleared")

print("\n" + "="*80)
print("✅ PHASE 1A: CPU SERIAL BASELINE COMPLETE")
print(f"   Baseline Time: {phase1a_total:.2f}s ({phase1a_total/60:.2f} minutes)")
print("="*80)

In [ ]:
# ============================================================================
# PHASE 1B: CPU DASK PARALLEL
# ============================================================================

import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import time
import json
import os
from datetime import datetime
import gc
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("PHASE 1B: CPU DASK PARALLEL")
print("NYC REAL ESTATE PROJECT")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Reset threading environment
for var in ['OMP_NUM_THREADS', 'MKL_NUM_THREADS', 'OPENBLAS_NUM_THREADS', 'NUMEXPR_NUM_THREADS']:
    if var in os.environ:
        del os.environ[var]

print("\n✓ Environment reset for parallel execution")

# Initialize Dask Client
print("\n📊 Initializing Dask Cluster...")
print("   Creating 16 workers with 4 threads each (64 total threads)")

cluster = LocalCluster(
    n_workers=16,
    threads_per_worker=4,
    processes=False,
    memory_limit='16GB',
    silence_logs=True
)
client = Client(cluster)

print(f"\n✓ Dask Client initialized")
print(f"✓ Workers: {len(client.scheduler_info()['workers'])}")
print(f"✓ Total threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")
print(f"✓ Dashboard: {client.dashboard_link}")

# Initialize timing dictionary
timings = {}

# START PHASE 1B TIMER
phase1b_total_start = time.time()

# STEP 1: DATA LOADING
print("\n" + "="*80)
print("STEP 1: DATA LOADING (DASK)")
print("="*80)
step_start = time.time()

print("Loading with Dask - all columns as strings first...")

ddf_1b = dd.read_csv(
    'nyc-property-sales.csv',
    blocksize='25MB',
    dtype=str,
    assume_missing=True
)

# FIX #1: Clean column names
ddf_1b.columns = ddf_1b.columns.str.strip()

# Trigger computation to measure load time
n_rows = len(ddf_1b)

timings['data_loading'] = time.time() - step_start
print(f"✓ Loaded {n_rows:,} rows and {len(ddf_1b.columns)} columns")
print(f"✓ Partitions: {ddf_1b.npartitions}")
print(f"⏱️  Time: {timings['data_loading']:.2f}s")

# STEP 2: FILTERING
print("\n" + "="*80)
print("STEP 2: PREPROCESSING (FILTERING)")
print("="*80)
step_start = time.time()

initial_rows = n_rows
print(f"Initial rows: {initial_rows:,}")

# Convert SALE PRICE to numeric
print("\nConverting SALE PRICE to numeric...")
ddf_1b['SALE PRICE'] = dd.to_numeric(ddf_1b['SALE PRICE'], errors='coerce')

# Remove invalid prices
ddf_1b = ddf_1b.dropna(subset=['SALE PRICE'])

# Remove zero-dollar sales
ddf_1b = ddf_1b[ddf_1b['SALE PRICE'] > 0]

# Filter price range ($100K - $20M)
ddf_1b = ddf_1b[(ddf_1b['SALE PRICE'] >= 100000) & (ddf_1b['SALE PRICE'] <= 20000000)]

# Trigger computation
final_rows = len(ddf_1b)

timings['filtering'] = time.time() - step_start
rows_removed = initial_rows - final_rows
print(f"After filtering: {final_rows:,} rows")
print(f"Rows removed: {rows_removed:,} ({rows_removed/initial_rows*100:.2f}%)")
print(f"⏱️  Time: {timings['filtering']:.2f}s")

# STEP 3: MISSING VALUES
print("\n" + "="*80)
print("STEP 3: HANDLE MISSING VALUES")
print("="*80)
step_start = time.time()

# Convert numeric columns from string
print("Converting numeric columns...")
ddf_1b['LAND SQUARE FEET'] = dd.to_numeric(ddf_1b['LAND SQUARE FEET'], errors='coerce')
ddf_1b['GROSS SQUARE FEET'] = dd.to_numeric(ddf_1b['GROSS SQUARE FEET'], errors='coerce')
ddf_1b['YEAR BUILT'] = dd.to_numeric(ddf_1b['YEAR BUILT'], errors='coerce')

# Use MEAN instead of median (works reliably with Dask)
print("Computing means in parallel for missing value imputation...")
print("Note: Using mean instead of median for Dask compatibility")
mean_land = ddf_1b['LAND SQUARE FEET'].mean().compute()
mean_gross = ddf_1b['GROSS SQUARE FEET'].mean().compute()
mean_year = ddf_1b['YEAR BUILT'].mean().compute()

ddf_1b['LAND SQUARE FEET'] = ddf_1b['LAND SQUARE FEET'].fillna(mean_land)
ddf_1b['GROSS SQUARE FEET'] = ddf_1b['GROSS SQUARE FEET'].fillna(mean_gross)
ddf_1b['YEAR BUILT'] = ddf_1b['YEAR BUILT'].fillna(mean_year)

timings['missing_values'] = time.time() - step_start
print(f"✓ LAND SQUARE FEET: filled with mean {mean_land:,.0f}")
print(f"✓ GROSS SQUARE FEET: filled with mean {mean_gross:,.0f}")
print(f"✓ YEAR BUILT: filled with mean {mean_year:.0f}")
print(f"⏱️  Time: {timings['missing_values']:.2f}s")

# STEP 4: DATE CONVERSION
print("\n" + "="*80)
print("STEP 4: DATE CONVERSION")
print("="*80)
step_start = time.time()

ddf_1b['SALE DATE'] = dd.to_datetime(ddf_1b['SALE DATE'], errors='coerce')

timings['date_conversion'] = time.time() - step_start
print(f"✓ Converted SALE DATE to datetime")
print(f"⏱️  Time: {timings['date_conversion']:.2f}s")

# STEP 5: SAVE DATA
print("\n" + "="*80)
print("STEP 5: SAVE PREPROCESSED DATA")
print("="*80)
output_path = 'data/processed/preprocessed_cpu_dask.csv'
os.makedirs('data/processed', exist_ok=True)

print("Saving to CSV (computing Dask dataframe first)...")
step_start = time.time()

# FIX #3: Use compute() then save (not deprecated single_file)
df_computed = ddf_1b.compute()
df_computed.to_csv(output_path, index=False)

timings['save_data'] = time.time() - step_start

output_size = os.path.getsize(output_path) / (1024**2)
print(f"✓ Saved to: {output_path}")
print(f"✓ Output size: {output_size:.2f} MB")
print(f"⏱️  Time: {timings['save_data']:.2f}s")

# END PHASE 1B TIMER
phase1b_total = time.time() - phase1b_total_start
timings['total'] = phase1b_total

# FINAL SUMMARY
print("\n" + "="*80)
print("PHASE 1B COMPLETE!")
print("="*80)
print(f"Final dataset: {final_rows:,} rows")
print(f"\n⏱️  TOTAL PHASE 1B TIME: {phase1b_total:.2f}s ({phase1b_total/60:.2f} minutes)")

# Timing breakdown
print("\n📊 Phase 1B Timing Breakdown:")
for op, t in timings.items():
    if op != 'total':
        print(f"  • {op:20s}: {t:8.2f}s")
print(f"  {'─'*35}")
print(f"  • {'TOTAL':20s}: {phase1b_total:8.2f}s")

# SPEEDUP CALCULATION
print("\n" + "="*80)
print("📊 PERFORMANCE COMPARISON: 1B vs 1A")
print("="*80)

# FIX #2: Load Phase 1A time from JSON instead of hardcoding
try:
    with open('results/timings/phase1a_timings.json', 'r') as f:
        phase1a_data = json.load(f)
        phase1a_time = phase1a_data['timings']['total']
    print(f"✓ Loaded Phase 1A baseline time: {phase1a_time:.2f}s")
except FileNotFoundError:
    print("⚠️  Warning: phase1a_timings.json not found, using default 29.03s")
    phase1a_time = 29.03

speedup = phase1a_time / phase1b_total
efficiency = (speedup / 64) * 100  # 64 threads

print(f"\nPhase 1A (CPU Serial):     {phase1a_time:.2f}s")
print(f"Phase 1B (CPU Dask):       {phase1b_total:.2f}s")
print(f"Speedup:                   {speedup:.2f}x")
print(f"Parallel Efficiency:       {efficiency:.1f}%")

if speedup < 1.0:
    time_diff = phase1b_total - phase1a_time
    print(f"Slowdown:                  {time_diff:.2f}s slower than serial")
    print("\n💡 Analysis: Slowdown is expected because:")
    print("   • CSV I/O is inherently sequential")
    print("   • Dask overhead exceeds parallelization benefits")
    print("   • Python GIL limits thread-based parallelism")
    print("   • Data loading dominates over computation")
    print("   • This demonstrates the challenge of parallelizing I/O-bound tasks")
else:
    time_saved = phase1a_time - phase1b_total
    print(f"Time saved:                {time_saved:.2f}s")
    if speedup < 2:
        print("\n💡 Analysis: Modest speedup is expected due to I/O bottleneck")
    else:
        print("\n✓ Good speedup achieved through parallel processing!")

print("\n📝 Note: Used mean (not median) for Dask compatibility")

# Save timings
timings_output = {
    'phase': 'phase1b_cpu_dask',
    'timestamp': datetime.now().isoformat(),
    'timings': timings,
    'dask_config': {
        'n_workers': 16,
        'threads_per_worker': 4,
        'total_threads': 64,
        'blocksize': '25MB',
        'imputation_method': 'mean'
    },
    'comparison': {
        'phase1a_time': float(phase1a_time),
        'phase1b_time': float(phase1b_total),
        'speedup': float(speedup),
        'efficiency_percent': float(efficiency)
    },
    'dataset_info': {
        'final_rows': int(final_rows),
        'output_size_mb': float(output_size)
    },
    'notes': 'Used mean instead of median for missing value imputation due to Dask limitations with median_approximate()'
}

os.makedirs('results/timings', exist_ok=True)
with open('results/timings/phase1b_timings.json', 'w') as f:
    json.dump(timings_output, f, indent=2)
print(f"\n✓ Timings saved to: results/timings/phase1b_timings.json")

# Cleanup
print("\n🧹 Cleaning up...")
client.close()
cluster.close()
del ddf_1b, df_computed
gc.collect()
print("✓ Dask client closed")
print("✓ Memory cleared")

print("\n" + "="*80)
print("✅ PHASE 1B: CPU DASK PARALLEL COMPLETE")
print(f"✅ Time: {phase1b_total:.2f}s | Speedup: {speedup:.2f}x")
print("="*80)
print("\n🚀 Ready for Phase 1C (CPU joblib)")

In [ ]:
# ============================================================================
# PHASE 1C: CPU PARALLEL - JOBLIB WITH PROCESSES
# ============================================================================

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import time
import json
import os
from datetime import datetime
import gc
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("PHASE 1C: CPU PARALLEL - JOBLIB")
print("NYC REAL ESTATE PROJECT")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Initialize timing dictionary
timings = {}
n_jobs = 16  # Number of parallel processes

print(f"\n✓ Using joblib with {n_jobs} parallel processes")
print("✓ This uses TRUE parallelism (multiprocessing, not threads)")
print("✓ No Python GIL limitations!")

# ====================
# START PHASE 1C TIMER
# ====================
phase1c_total_start = time.time()

# STEP 1: DATA LOADING (Read CSV in chunks, process in parallel)
print("\n" + "="*80)
print("STEP 1: DATA LOADING (PARALLEL CHUNKS)")
print("="*80)
step_start = time.time()

# FIX #1: Use current directory path (consistent with 1A/1B)
file_path = 'nyc-property-sales.csv'

# Strategy: Read file in parallel chunks
print("Reading CSV in parallel chunks...")

# First, read header to get column names
df_header = pd.read_csv(file_path, nrows=0)
columns = df_header.columns.tolist()

# Count total rows (fast approximation)
print("Determining file size...")
with open(file_path, 'r') as f:
    total_lines = sum(1 for _ in f) - 1  # -1 for header

print(f"Total rows to process: {total_lines:,}")

# Calculate chunk size
chunk_size = total_lines // n_jobs + 1
print(f"Chunk size per process: {chunk_size:,} rows")

# Function to read a chunk
def read_chunk(skip_rows, nrows):
    """Read a chunk of the CSV file"""
    return pd.read_csv(
        file_path,
        skiprows=range(1, skip_rows) if skip_rows > 0 else None,
        nrows=nrows,
        low_memory=False
    )

# Create chunk parameters
chunk_params = []
for i in range(n_jobs):
    skip = i * chunk_size + 1  # +1 to skip header
    if i == 0:
        skip = 0  # First chunk doesn't skip header
    nrows = min(chunk_size, total_lines - i * chunk_size)
    if nrows > 0:
        chunk_params.append((skip, nrows))

print(f"Created {len(chunk_params)} chunks for parallel processing")

# Read chunks in parallel
print("Reading chunks in parallel...")
chunks = Parallel(n_jobs=n_jobs, verbose=5)(
    delayed(read_chunk)(skip, nrows) for skip, nrows in chunk_params
)

# Concatenate all chunks
print("\nConcatenating chunks...")
df_1c = pd.concat(chunks, ignore_index=True)
del chunks
gc.collect()

# FIX #3: Clean column names after concatenation
df_1c.columns = df_1c.columns.str.strip()

timings['data_loading'] = time.time() - step_start
print(f"\n✓ Loaded {len(df_1c):,} rows and {len(df_1c.columns)} columns")
print(f"✓ Memory usage: {df_1c.memory_usage(deep=True).sum() / (1024**3):.2f} GB")
print(f"⏱️  Time: {timings['data_loading']:.2f}s")

# STEP 2: FILTERING
print("\n" + "="*80)
print("STEP 2: PREPROCESSING (FILTERING)")
print("="*80)
step_start = time.time()

initial_rows = len(df_1c)
print(f"Initial rows: {initial_rows:,}")

# Convert SALE PRICE to numeric
print("\nConverting SALE PRICE to numeric...")
df_1c['SALE PRICE'] = pd.to_numeric(df_1c['SALE PRICE'], errors='coerce')
df_1c = df_1c.dropna(subset=['SALE PRICE'])

# Remove zero-dollar sales
df_1c = df_1c[df_1c['SALE PRICE'] > 0].copy()

# Filter price range
df_1c = df_1c[(df_1c['SALE PRICE'] >= 100000) & (df_1c['SALE PRICE'] <= 20000000)].copy()

timings['filtering'] = time.time() - step_start
final_rows = len(df_1c)
rows_removed = initial_rows - final_rows
print(f"After filtering: {final_rows:,} rows")
print(f"Rows removed: {rows_removed:,} ({rows_removed/initial_rows*100:.2f}%)")
print(f"⏱️  Time: {timings['filtering']:.2f}s")

# STEP 3: MISSING VALUES
print("\n" + "="*80)
print("STEP 3: HANDLE MISSING VALUES")
print("="*80)
step_start = time.time()

# Convert numeric columns
df_1c['LAND SQUARE FEET'] = pd.to_numeric(df_1c['LAND SQUARE FEET'], errors='coerce')
df_1c['GROSS SQUARE FEET'] = pd.to_numeric(df_1c['GROSS SQUARE FEET'], errors='coerce')
df_1c['YEAR BUILT'] = pd.to_numeric(df_1c['YEAR BUILT'], errors='coerce')

# Compute medians (same as Phase 1A for fair comparison)
median_land = df_1c['LAND SQUARE FEET'].median()
median_gross = df_1c['GROSS SQUARE FEET'].median()
median_year = df_1c['YEAR BUILT'].median()

df_1c['LAND SQUARE FEET'].fillna(median_land, inplace=True)
df_1c['GROSS SQUARE FEET'].fillna(median_gross, inplace=True)
df_1c['YEAR BUILT'].fillna(median_year, inplace=True)

timings['missing_values'] = time.time() - step_start
print(f"✓ LAND SQUARE FEET: filled with median {median_land:,.0f}")
print(f"✓ GROSS SQUARE FEET: filled with median {median_gross:,.0f}")
print(f"✓ YEAR BUILT: filled with median {median_year:.0f}")
print(f"⏱️  Time: {timings['missing_values']:.2f}s")

# STEP 4: DATE CONVERSION
print("\n" + "="*80)
print("STEP 4: DATE CONVERSION")
print("="*80)
step_start = time.time()

df_1c['SALE DATE'] = pd.to_datetime(df_1c['SALE DATE'], errors='coerce')

timings['date_conversion'] = time.time() - step_start
print(f"✓ Converted SALE DATE to datetime")
print(f"⏱️  Time: {timings['date_conversion']:.2f}s")

# STEP 5: SAVE DATA
print("\n" + "="*80)
print("STEP 5: SAVE PREPROCESSED DATA")
print("="*80)
# FIX #4: Current directory path (consistent with 1A/1B)
output_path = 'data/processed/preprocessed_cpu_joblib.csv'
os.makedirs('data/processed', exist_ok=True)

step_start = time.time()
df_1c.to_csv(output_path, index=False)
timings['save_data'] = time.time() - step_start

output_size = os.path.getsize(output_path) / (1024**2)
print(f"✓ Saved to: {output_path}")
print(f"✓ Output size: {output_size:.2f} MB")
print(f"⏱️  Time: {timings['save_data']:.2f}s")

# ====================
# END PHASE 1C TIMER
# ====================
phase1c_total = time.time() - phase1c_total_start
timings['total'] = phase1c_total

# FINAL SUMMARY
print("\n" + "="*80)
print("PHASE 1C COMPLETE!")
print("="*80)
print(f"Final dataset: {final_rows:,} rows")
print(f"\n⏱️  TOTAL PHASE 1C TIME: {phase1c_total:.2f}s ({phase1c_total/60:.2f} minutes)")

# Timing breakdown
print("\n📊 Phase 1C Timing Breakdown:")
for op, t in timings.items():
    if op != 'total':
        print(f"  • {op:20s}: {t:8.2f}s")
print(f"  {'─'*35}")
print(f"  • {'TOTAL':20s}: {phase1c_total:8.2f}s")

# SPEEDUP CALCULATION
print("\n" + "="*80)
print("📊 PERFORMANCE COMPARISON: ALL CPU PHASES")
print("="*80)

# FIX #2: Load times from JSON files instead of hardcoding
try:
    with open('results/timings/phase1a_timings.json', 'r') as f:
        phase1a_data = json.load(f)
        phase1a_time = phase1a_data['timings']['total']
    print(f"✓ Loaded Phase 1A time: {phase1a_time:.2f}s")
except FileNotFoundError:
    print("⚠️  Warning: phase1a_timings.json not found, using default 29.03s")
    phase1a_time = 29.03

try:
    with open('results/timings/phase1b_timings.json', 'r') as f:
        phase1b_data = json.load(f)
        phase1b_time = phase1b_data['timings']['total']
    print(f"✓ Loaded Phase 1B time: {phase1b_time:.2f}s")
except FileNotFoundError:
    print("⚠️  Warning: phase1b_timings.json not found, using default 91.85s")
    phase1b_time = 91.85

speedup_vs_1a = phase1a_time / phase1c_total
speedup_vs_1b = phase1b_time / phase1c_total
efficiency = (speedup_vs_1a / n_jobs) * 100

print(f"\nPhase 1A (CPU Serial):     {phase1a_time:.2f}s")
print(f"Phase 1B (CPU Dask):       {phase1b_time:.2f}s")
print(f"Phase 1C (CPU joblib):     {phase1c_total:.2f}s")
print(f"\nSpeedup vs 1A (Serial):    {speedup_vs_1a:.2f}x")
print(f"Speedup vs 1B (Dask):      {speedup_vs_1b:.2f}x")
print(f"Parallel Efficiency:       {efficiency:.1f}%")

if speedup_vs_1a >= 1.0:
    time_saved = phase1a_time - phase1c_total
    print(f"Time saved vs Serial:      {time_saved:.2f}s")
else:
    time_lost = phase1c_total - phase1a_time
    print(f"Time lost vs Serial:       {time_lost:.2f}s")

if speedup_vs_1a >= 2:
    print("\n🎉 Excellent! joblib achieved good speedup with true parallelism!")
elif speedup_vs_1a >= 1.5:
    print("\n✓ Good speedup achieved - process-based parallelism works!")
elif speedup_vs_1a > 1:
    print("\n✓ Moderate speedup - some benefit from parallel processing")
else:
    print("\n⚠️  Limited speedup - CSV I/O remains the bottleneck")
    print("💡 Analysis: Even with true parallelism (no GIL):")
    print("   • CSV I/O is inherently sequential")
    print("   • File system becomes the bottleneck")
    print("   • Parallelization overhead can exceed benefits")

print("\n📊 Key Insights:")
print(f"  • joblib (processes) vs Dask (threads): {speedup_vs_1b:.2f}x faster")
if speedup_vs_1b > 1:
    print(f"  • True parallelism (no GIL) makes a difference!")
print(f"  • Data loading was parallelized by reading chunks concurrently")

# Save timings
timings_output = {
    'phase': 'phase1c_cpu_joblib',
    'timestamp': datetime.now().isoformat(),
    'timings': timings,
    'joblib_config': {
        'n_jobs': n_jobs,
        'backend': 'multiprocessing',
        'chunk_strategy': 'parallel_read_chunks'
    },
    'comparison': {
        'phase1a_time': float(phase1a_time),
        'phase1b_time': float(phase1b_time),
        'phase1c_time': float(phase1c_total),
        'speedup_vs_serial': float(speedup_vs_1a),
        'speedup_vs_dask': float(speedup_vs_1b),
        'efficiency_percent': float(efficiency)
    },
    'dataset_info': {
        'final_rows': int(final_rows),
        'output_size_mb': float(output_size)
    }
}

os.makedirs('results/timings', exist_ok=True)
with open('results/timings/phase1c_timings.json', 'w') as f:
    json.dump(timings_output, f, indent=2)
print(f"\n✓ Timings saved to: results/timings/phase1c_timings.json")

# Cleanup
print("\n🧹 Cleaning up...")
del df_1c
gc.collect()
print("✓ Memory cleared")

print("\n" + "="*80)
print("✅ PHASE 1C: CPU PARALLEL - JOBLIB COMPLETE")
print(f"✅ Time: {phase1c_total:.2f}s | Speedup: {speedup_vs_1a:.2f}x")
print("="*80)
print("\n" + "="*80)
print("CPU PHASES COMPLETE - READY FOR GPU PHASES!")
print("="*80)
print("\n📊 Summary of CPU Phases:")
print(f"  Phase 1A (Serial):  {phase1a_time:.2f}s  [baseline]")
print(f"  Phase 1B (Dask):    {phase1b_time:.2f}s  [{phase1a_time/phase1b_time:.2f}x]")
print(f"  Phase 1C (joblib):  {phase1c_total:.2f}s  [{speedup_vs_1a:.2f}x]")
print("\n🚀 Next: GPU acceleration phases (cuDF, Dask-cuDF)")

In [ ]:
# ============================================================================
# PHASE 2: FEATURE ENGINEERING - CPU SERIAL
# ============================================================================

import pandas as pd
import numpy as np
import time
import json
import os
from datetime import datetime
import gc

print("="*80)
print("PHASE 2: FEATURE ENGINEERING")
print("NYC REAL ESTATE PROJECT")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Working directory: {os.getcwd()}")

# Initialize timing dictionary
timings = {}

# ====================
# START PHASE 2 TIMER
# ====================
phase2_total_start = time.time()

# STEP 1: LOAD PREPROCESSED DATA
print("\n" + "="*80)
print("STEP 1: LOAD PREPROCESSED DATA (FROM PHASE 1A)")
print("="*80)
step_start = time.time()

# FIX #1 & #2: Current directory path, load from Phase 1A (serial baseline)
input_path = 'data/processed/preprocessed_cpu_serial.csv'
print(f"Loading from: {input_path}")

df = pd.read_csv(input_path, low_memory=False)

# Clean column names
df.columns = df.columns.str.strip()
print(f"✓ Cleaned column names (removed leading/trailing spaces)")

timings['data_loading'] = time.time() - step_start
print(f"✓ Loaded {len(df):,} rows and {len(df.columns)} columns")
print(f"✓ Memory usage: {df.memory_usage(deep=True).sum() / (1024**3):.2f} GB")
print(f"  Time: {timings['data_loading']:.2f}s")

# Ensure SALE DATE is datetime
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])

# STEP 2: TEMPORAL FEATURES
print("\n" + "="*80)
print("STEP 2: CREATE TEMPORAL FEATURES")
print("="*80)
step_start = time.time()

print("Extracting temporal features from SALE DATE...")

# Extract date components
df['SALE_YEAR'] = df['SALE DATE'].dt.year
df['SALE_MONTH'] = df['SALE DATE'].dt.month
df['SALE_QUARTER'] = df['SALE DATE'].dt.quarter
df['SALE_DAY_OF_WEEK'] = df['SALE DATE'].dt.dayofweek
df['SALE_DAY_OF_YEAR'] = df['SALE DATE'].dt.dayofyear

print("✓ Created: SALE_YEAR, SALE_MONTH, SALE_QUARTER, SALE_DAY_OF_WEEK, SALE_DAY_OF_YEAR")

# Calculate property age at time of sale
print("\nCalculating property age...")
df['PROPERTY_AGE'] = df['SALE_YEAR'] - df['YEAR BUILT']

# Handle negative ages (data errors)
negative_ages = (df['PROPERTY_AGE'] < 0).sum()
if negative_ages > 0:
    print(f"    Found {negative_ages:,} properties with negative age (data errors)")
    df.loc[df['PROPERTY_AGE'] < 0, 'PROPERTY_AGE'] = 0

print(f"✓ Created: PROPERTY_AGE")
print(f"  Age range: {df['PROPERTY_AGE'].min():.0f} to {df['PROPERTY_AGE'].max():.0f} years")

timings['temporal_features'] = time.time() - step_start
print(f"  Time: {timings['temporal_features']:.2f}s")

# STEP 3: SIZE-BASED FEATURES
print("\n" + "="*80)
print("STEP 3: CREATE SIZE-BASED FEATURES")
print("="*80)
step_start = time.time()

# Price per square foot (key real estate metric)
print("Calculating price per square foot...")
df['PRICE_PER_SQFT'] = df['SALE PRICE'] / df['GROSS SQUARE FEET']

# Handle infinite/NaN values (where GROSS SQUARE FEET = 0)
inf_count = np.isinf(df['PRICE_PER_SQFT']).sum()
if inf_count > 0:
    print(f"  Found {inf_count:,} infinite values (GROSS SQUARE FEET = 0)")
    df.loc[np.isinf(df['PRICE_PER_SQFT']), 'PRICE_PER_SQFT'] = np.nan
    median_price_sqft = df['PRICE_PER_SQFT'].median()
    df['PRICE_PER_SQFT'].fillna(median_price_sqft, inplace=True)
    print(f"  Filled with median: ${median_price_sqft:.2f}/sqft")

print(f"✓ Created: PRICE_PER_SQFT")
print(f"  Range: ${df['PRICE_PER_SQFT'].min():.2f} to ${df['PRICE_PER_SQFT'].max():.2f} per sqft")

# Price per unit (for multi-unit buildings)
print("\nCalculating price per unit...")
df['PRICE_PER_UNIT'] = df['SALE PRICE'] / df['TOTAL UNITS']

# Handle division by zero
inf_count = np.isinf(df['PRICE_PER_UNIT']).sum()
if inf_count > 0:
    df.loc[np.isinf(df['PRICE_PER_UNIT']) | (df['TOTAL UNITS'] == 0), 'PRICE_PER_UNIT'] = df['SALE PRICE']

print(f"✓ Created: PRICE_PER_UNIT")

# Land to building ratio
print("\nCalculating land-to-building ratio...")
df['LAND_TO_BUILDING_RATIO'] = df['LAND SQUARE FEET'] / df['GROSS SQUARE FEET']
df.loc[np.isinf(df['LAND_TO_BUILDING_RATIO']), 'LAND_TO_BUILDING_RATIO'] = 1.0

print(f"✓ Created: LAND_TO_BUILDING_RATIO")

# Commercial ratio (for mixed-use properties)
print("\nCalculating commercial ratio...")
df['COMMERCIAL_RATIO'] = df['COMMERCIAL UNITS'] / df['TOTAL UNITS']
df.loc[np.isinf(df['COMMERCIAL_RATIO']) | df['COMMERCIAL_RATIO'].isna(), 'COMMERCIAL_RATIO'] = 0.0

print(f"✓ Created: COMMERCIAL_RATIO")

timings['size_features'] = time.time() - step_start
print(f"  Time: {timings['size_features']:.2f}s")

# STEP 4: LOCATION-BASED AGGREGATIONS
print("\n" + "="*80)
print("STEP 4: CREATE LOCATION-BASED FEATURES")
print("="*80)
step_start = time.time()

print("Computing average prices by location...")
print("Note: This is where GPU would help with high-cardinality groupby!")

# Average price by BOROUGH
print("\n1. By BOROUGH...")
borough_avg = df.groupby('BOROUGH')['SALE PRICE'].mean()
df['BOROUGH_AVG_PRICE'] = df['BOROUGH'].map(borough_avg)
print(f"   ✓ {len(borough_avg)} boroughs")

# Average price by NEIGHBORHOOD (high cardinality!)
print("\n2. By NEIGHBORHOOD...")
neighborhood_avg = df.groupby('NEIGHBORHOOD')['SALE PRICE'].mean()
df['NEIGHBORHOOD_AVG_PRICE'] = df['NEIGHBORHOOD'].map(neighborhood_avg)
print(f"   ✓ {len(neighborhood_avg)} neighborhoods")

# Average price by ZIP CODE
print("\n3. By ZIP CODE...")
zip_avg = df.groupby('ZIP CODE')['SALE PRICE'].mean()
df['ZIP_AVG_PRICE'] = df['ZIP CODE'].map(zip_avg)
print(f"   ✓ {len(zip_avg)} zip codes")

# Average price by BUILDING CLASS CATEGORY
print("\n4. By BUILDING CLASS CATEGORY...")
building_class_avg = df.groupby('BUILDING CLASS CATEGORY')['SALE PRICE'].mean()
df['BUILDING_CLASS_AVG_PRICE'] = df['BUILDING CLASS CATEGORY'].map(building_class_avg)
print(f"   ✓ {len(building_class_avg)} building classes")

timings['location_aggregations'] = time.time() - step_start
print(f"  Time: {timings['location_aggregations']:.2f}s")

# STEP 5: DERIVED PRICE FEATURES
print("\n" + "="*80)
print("STEP 5: CREATE DERIVED PRICE FEATURES")
print("="*80)
step_start = time.time()

# Price deviation from neighborhood average
print("Calculating price deviations from local averages...")
df['PRICE_VS_NEIGHBORHOOD'] = (df['SALE PRICE'] - df['NEIGHBORHOOD_AVG_PRICE']) / df['NEIGHBORHOOD_AVG_PRICE']
df['PRICE_VS_BOROUGH'] = (df['SALE PRICE'] - df['BOROUGH_AVG_PRICE']) / df['BOROUGH_AVG_PRICE']
df['PRICE_VS_ZIP'] = (df['SALE PRICE'] - df['ZIP_AVG_PRICE']) / df['ZIP_AVG_PRICE']

print("✓ Created: PRICE_VS_NEIGHBORHOOD, PRICE_VS_BOROUGH, PRICE_VS_ZIP")

# Flag undervalued properties (below neighborhood average)
df['IS_UNDERVALUED'] = (df['SALE PRICE'] < df['NEIGHBORHOOD_AVG_PRICE']).astype(int)
undervalued_count = df['IS_UNDERVALUED'].sum()
print(f"✓ Created: IS_UNDERVALUED flag ({undervalued_count:,} properties flagged)")

timings['derived_features'] = time.time() - step_start
print(f"  Time: {timings['derived_features']:.2f}s")

# STEP 6: SAVE ENGINEERED FEATURES
print("\n" + "="*80)
print("STEP 6: SAVE FEATURE-ENGINEERED DATA")
print("="*80)
# FIX #3: Current directory path
output_path = 'data/processed/feature_engineered.csv'
os.makedirs('data/processed', exist_ok=True)

step_start = time.time()
df.to_csv(output_path, index=False)
timings['save_data'] = time.time() - step_start

output_size = os.path.getsize(output_path) / (1024**2)
print(f"✓ Saved to: {output_path}")
print(f"✓ Output size: {output_size:.2f} MB")
print(f"  Time: {timings['save_data']:.2f}s")

# ====================
# END PHASE 2 TIMER
# ====================
phase2_total = time.time() - phase2_total_start
timings['total'] = phase2_total

# SUMMARY
print("\n" + "="*80)
print("PHASE 2 COMPLETE!")
print("="*80)
print(f"Final dataset: {len(df):,} rows, {len(df.columns)} columns")
print(f"\n⏱️  TOTAL PHASE 2 TIME: {phase2_total:.2f}s ({phase2_total/60:.2f} minutes)")

# Show new features created
new_features = [
    'SALE_YEAR', 'SALE_MONTH', 'SALE_QUARTER', 'SALE_DAY_OF_WEEK', 'SALE_DAY_OF_YEAR',
    'PROPERTY_AGE', 'PRICE_PER_SQFT', 'PRICE_PER_UNIT', 'LAND_TO_BUILDING_RATIO',
    'COMMERCIAL_RATIO', 'BOROUGH_AVG_PRICE', 'NEIGHBORHOOD_AVG_PRICE', 
    'ZIP_AVG_PRICE', 'BUILDING_CLASS_AVG_PRICE', 'PRICE_VS_NEIGHBORHOOD',
    'PRICE_VS_BOROUGH', 'PRICE_VS_ZIP', 'IS_UNDERVALUED'
]

print(f"\n📊 Features Created: {len(new_features)}")
for i, feat in enumerate(new_features, 1):
    print(f"  {i:2d}. {feat}")

# Timing breakdown
print("\n📊 Phase 2 Timing Breakdown:")
for op, t in timings.items():
    if op != 'total':
        print(f"  • {op:25s}: {t:8.2f}s")
print(f"  {'─'*40}")
print(f"  • {'TOTAL':25s}: {phase2_total:8.2f}s")

# Save timings
timings_output = {
    'phase': 'phase2_feature_engineering',
    'timestamp': datetime.now().isoformat(),
    'timings': timings,
    'features_created': new_features,
    'dataset_info': {
        'rows': int(len(df)),
        'columns': int(len(df.columns)),
        'original_columns': 21,
        'new_features': len(new_features),
        'total_features': int(len(df.columns)),
        'output_size_mb': float(output_size)
    }
}

# FIX #4: Current directory path
os.makedirs('results/timings', exist_ok=True)
with open('results/timings/phase2_timings.json', 'w') as f:
    json.dump(timings_output, f, indent=2)
print(f"\n✓ Timings saved to: results/timings/phase2_timings.json")

# Show sample of new features
print("\n" + "="*80)
print("SAMPLE OF ENGINEERED FEATURES (First 5 rows)")
print("="*80)
sample_cols = ['SALE PRICE', 'PROPERTY_AGE', 'PRICE_PER_SQFT', 'NEIGHBORHOOD_AVG_PRICE', 'IS_UNDERVALUED']
print(df[sample_cols].head().to_string())

# Clear memory
print("\n🧹 Clearing memory...")
del df
gc.collect()
print("✓ Memory cleared")

print("\n" + "="*80)
print("✅ PHASE 2: FEATURE ENGINEERING COMPLETE")
print(f"✅ Time: {phase2_total:.2f}s ({phase2_total/60:.2f} minutes)")
print("="*80)
print("\n🚀 Next: Phase 3A/3B - XGBoost CPU Training (Serial & Parallel)")

In [ ]:
%%time
# ============================================================================
# PHASE 3: MODEL TRAINING - CPU (SERIAL & PARALLEL WITH FULL SCALING)
# Tests: 1, 4, 8, 16, 32, 56 threads to find optimal scaling
# ============================================================================

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import time
import json
import os
from datetime import datetime
import gc
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("PHASE 3: MODEL TRAINING - CPU (SERIAL & PARALLEL)")
print("NYC REAL ESTATE PROJECT")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Track overall phase time
phase3_overall_start = time.time()

# ====================
# STEP 1: LOAD FEATURE-ENGINEERED DATA
# ====================
print("\n" + "="*80)
print("STEP 1: LOAD FEATURE-ENGINEERED DATA")
print("="*80)

step_start = time.time()
# FIX #1: Current directory path
input_path = 'data/processed/feature_engineered.csv'
print(f"Loading from: {input_path}")

df = pd.read_csv(input_path, low_memory=False)

# FIX #2: Verify column names are cleaned
df.columns = df.columns.str.strip()
print("✓ Verified column names are clean")

load_time = time.time() - step_start
print(f"✓ Loaded {len(df):,} rows and {len(df.columns)} columns")
print(f"  Time: {load_time:.2f}s")

# Convert SALE DATE to datetime
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])

# ====================
# STEP 2: TIME-AWARE TRAIN/VAL/TEST SPLIT
# ====================
print("\n" + "="*80)
print("STEP 2: TIME-AWARE DATA SPLIT")
print("="*80)
print("Following proposal: Train through 2021, Validation 2022, Test 2023")

step_start = time.time()

# Time-aware split (as per proposal)
train_df = df[df['SALE_YEAR'] <= 2021].copy()
val_df = df[df['SALE_YEAR'] == 2022].copy()
test_df = df[df['SALE_YEAR'] == 2023].copy()

split_time = time.time() - step_start

print(f"\n📊 Data Split:")
print(f"  Training (≤2021):   {len(train_df):,} rows ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Validation (2022):  {len(val_df):,} rows ({len(val_df)/len(df)*100:.1f}%)")
print(f"  Test (2023):        {len(test_df):,} rows ({len(test_df)/len(df)*100:.1f}%)")
print(f"  Time: {split_time:.2f}s")

# ====================
# STEP 3: FEATURE SELECTION & CATEGORICAL ENCODING
# ====================
print("\n" + "="*80)
print("STEP 3: FEATURE SELECTION & CATEGORICAL ENCODING")
print("="*80)

step_start = time.time()

# Select features for modeling
exclude_cols = ['SALE DATE', 'ADDRESS', 'APARTMENT NUMBER', 'SALE PRICE', 
                'BUILDING CLASS AT PRESENT', 'BUILDING CLASS AT TIME OF SALE',
                'TAX CLASS AT PRESENT', 'TAX CLASS AT TIME OF SALE']

feature_cols = [col for col in df.columns if col not in exclude_cols]
print(f"Total features for modeling: {len(feature_cols)}")

# Identify categorical columns
categorical_cols = ['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'ZIP CODE', 
                   'BLOCK', 'LOT', 'EASE-MENT']
categorical_cols = [col for col in categorical_cols if col in feature_cols]

print(f"\nCategorical features: {len(categorical_cols)}")
for cat in categorical_cols:
    n_unique = train_df[cat].nunique()
    print(f"  • {cat:30s} ({n_unique:,} unique values)")

# Encode categorical variables
print("\nEncoding categorical variables...")
label_encoders = {}
encoding_times = {}

for col in categorical_cols:
    col_start = time.time()
    
    le = LabelEncoder()
    
    # Convert to string
    train_df[col] = train_df[col].astype(str)
    val_df[col] = val_df[col].astype(str)
    test_df[col] = test_df[col].astype(str)
    
    # Fit on training data
    le.fit(train_df[col])
    
    # Transform training data
    train_df[col] = le.transform(train_df[col])
    
    # Create mapping dictionary for fast lookups
    class_to_index = {cls: idx for idx, cls in enumerate(le.classes_)}
    
    # Transform val/test with dictionary lookup
    val_df[col] = val_df[col].map(lambda x: class_to_index.get(x, -1))
    test_df[col] = test_df[col].map(lambda x: class_to_index.get(x, -1))
    
    label_encoders[col] = le
    encoding_times[col] = time.time() - col_start
    
    print(f"  ✓ {col:30s}: {encoding_times[col]:6.2f}s ({len(le.classes_):,} classes)")

total_encoding_time = sum(encoding_times.values())
print(f"\n  Total encoding time: {total_encoding_time:.2f}s ({total_encoding_time/60:.2f} min)")

# Prepare feature matrices
X_train = train_df[feature_cols]
y_train = train_df['SALE PRICE']

X_val = val_df[feature_cols]
y_val = val_df['SALE PRICE']

X_test = test_df[feature_cols]
y_test = test_df['SALE PRICE']

prep_time = time.time() - step_start

print(f"\n✓ Prepared feature matrices:")
print(f"  X_train: {X_train.shape}")
print(f"  X_val:   {X_val.shape}")
print(f"  X_test:  {X_test.shape}")
print(f"  Total Step 3 Time: {prep_time:.2f}s ({prep_time/60:.2f} min)")

# ====================
# PHASE 3: CPU SCALING EXPERIMENTS
# ====================
print("\n" + "="*80)
print("PHASE 3: CPU SCALING EXPERIMENTS")
print("="*80)
print("Testing thread counts: 1, 4, 8, 16, 32, 56")
print("This will take approximately 5-10 minutes total")

# Base parameters
base_params = {
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'verbosity': 0
}

# Thread counts to test
thread_counts = [1, 4, 8, 16, 32, 56]
all_results = []

# Run experiments for each thread count
for n_threads in thread_counts:
    print(f"\n{'='*80}")
    print(f"TESTING WITH {n_threads} THREAD{'S' if n_threads > 1 else ''}")
    print('='*80)
    
    # Set thread count
    params = base_params.copy()
    params['nthread'] = n_threads
    
    if n_threads == 1:
        print("Phase 3A: CPU Serial Baseline")
    else:
        print(f"Phase 3B: CPU Parallel ({n_threads} threads)")
    
    print(f"\n📊 Model Parameters:")
    print(f"  nthread: {n_threads}")
    print(f"  tree_method: hist")
    print(f"  n_estimators: 100")
    
    # Training
    print(f"\n⏳ Training XGBoost with {n_threads} thread{'s' if n_threads > 1 else ''}...")
    train_start = time.time()
    
    model = xgb.XGBRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    training_time = time.time() - train_start
    
    print(f"✓ Training complete: {training_time:.2f}s ({training_time/60:.2f} min)")
    
    # Verify model
    n_trees = len(model.get_booster().get_dump())
    print(f"✓ Model built: {n_trees} trees")
    
    # Prediction
    pred_start = time.time()
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    prediction_time = time.time() - pred_start
    
    print(f"✓ Predictions complete: {prediction_time:.2f}s")
    
    # Evaluation
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    
    val_mae = mean_absolute_error(y_val, y_val_pred)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    val_r2 = r2_score(y_val, y_val_pred)
    
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_r2 = r2_score(y_test, y_test_pred)
    
    print(f"\n📊 Model Performance:")
    print(f"  Test MAE:  ${test_mae:,.2f}")
    print(f"  Test RMSE: ${test_rmse:,.2f}")
    print(f"  Test R²:   {test_r2:.4f}")
    
    # Store results
    result = {
        'n_threads': n_threads,
        'training_time': training_time,
        'prediction_time': prediction_time,
        'total_time': training_time + prediction_time,
        'train_mae': float(train_mae),
        'train_rmse': float(train_rmse),
        'train_r2': float(train_r2),
        'val_mae': float(val_mae),
        'val_rmse': float(val_rmse),
        'val_r2': float(val_r2),
        'test_mae': float(test_mae),
        'test_rmse': float(test_rmse),
        'test_r2': float(test_r2),
        'n_trees': n_trees
    }
    all_results.append(result)
    
    # Save model for baseline (1 thread) and best parallel (56 threads)
    if n_threads == 1:
        model_path = 'models/xgboost_cpu_serial.json'
        os.makedirs('models', exist_ok=True)
        model.save_model(model_path)
        print(f"✓ Saved model: {model_path}")
    elif n_threads == 56:
        model_path = 'models/xgboost_cpu_parallel_56.json'
        os.makedirs('models', exist_ok=True)
        model.save_model(model_path)
        print(f"✓ Saved model: {model_path}")

# ====================
# PERFORMANCE ANALYSIS
# ====================
print("\n" + "="*80)
print("📊 COMPLETE PERFORMANCE ANALYSIS")
print("="*80)

# Get baseline (1 thread)
baseline_time = all_results[0]['training_time']

# Create analysis table
print("\n" + "="*80)
print("TRAINING TIME & SPEEDUP ANALYSIS")
print("="*80)

analysis_data = []
for result in all_results:
    n_threads = result['n_threads']
    train_time = result['training_time']
    speedup = baseline_time / train_time
    efficiency = (speedup / n_threads) * 100
    
    analysis_data.append({
        'Threads': n_threads,
        'Training Time (s)': f"{train_time:.2f}",
        'Speedup': f"{speedup:.2f}x",
        'Efficiency': f"{efficiency:.1f}%",
        'Test MAE': f"${result['test_mae']:,.2f}",
        'Test R²': f"{result['test_r2']:.4f}"
    })

analysis_df = pd.DataFrame(analysis_data)
print("\n" + analysis_df.to_string(index=False))

# Key findings
best_speedup_idx = max(range(len(all_results)), key=lambda i: baseline_time / all_results[i]['training_time'])
best_result = all_results[best_speedup_idx]

print(f"\n📈 KEY FINDINGS:")
print(f"  • Baseline (1 thread):     {baseline_time:.2f}s")
print(f"  • Best time ({best_result['n_threads']} threads):      {best_result['training_time']:.2f}s")
print(f"  • Maximum speedup:         {baseline_time/best_result['training_time']:.2f}x")
print(f"  • Model accuracy:          R²={best_result['test_r2']:.4f} (consistent across all)")

# Efficiency analysis
print(f"\n💡 EFFICIENCY ANALYSIS:")
for result in all_results[1:]:  # Skip baseline
    n_threads = result['n_threads']
    speedup = baseline_time / result['training_time']
    efficiency = (speedup / n_threads) * 100
    
    if efficiency >= 80:
        status = "Excellent"
    elif efficiency >= 60:
        status = "Good"
    elif efficiency >= 40:
        status = "Moderate"
    elif efficiency >= 20:
        status = "Limited"
    else:
        status = "Poor"
    
    print(f"  • {n_threads:2d} threads: {efficiency:5.1f}% efficiency - {status}")

print(f"\n📝 Note: Diminishing returns are expected (Amdahl's Law)")
print(f"  Sequential fraction limits parallel speedup")

# ====================
# SAVE ALL RESULTS
# ====================
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

# FIX #1: Current directory paths
os.makedirs('results/timings', exist_ok=True)
os.makedirs('models', exist_ok=True)

# Save Phase 3A (baseline)
phase3a_output = {
    'phase': 'phase3a_cpu_serial_xgboost',
    'timestamp': datetime.now().isoformat(),
    'timings': {
        'training': all_results[0]['training_time'],
        'prediction': all_results[0]['prediction_time'],
        'total': all_results[0]['total_time']
    },
    'model_params': {**base_params, 'nthread': 1},
    'metrics': {
        'train_mae': all_results[0]['train_mae'],
        'train_rmse': all_results[0]['train_rmse'],
        'train_r2': all_results[0]['train_r2'],
        'val_mae': all_results[0]['val_mae'],
        'val_rmse': all_results[0]['val_rmse'],
        'val_r2': all_results[0]['val_r2'],
        'test_mae': all_results[0]['test_mae'],
        'test_rmse': all_results[0]['test_rmse'],
        'test_r2': all_results[0]['test_r2']
    },
    'dataset_info': {
        'train_rows': len(X_train),
        'val_rows': len(X_val),
        'test_rows': len(X_test),
        'n_features': X_train.shape[1],
        'n_trees_built': all_results[0]['n_trees']
    }
}

with open('results/timings/phase3a_timings.json', 'w') as f:
    json.dump(phase3a_output, f, indent=2)
print("✓ Saved Phase 3A results: results/timings/phase3a_timings.json")

# Save complete CPU scaling results
cpu_scaling_output = {
    'phase': 'phase3_cpu_scaling_complete',
    'timestamp': datetime.now().isoformat(),
    'experiment': 'XGBoost CPU Thread Scaling',
    'thread_counts_tested': thread_counts,
    'baseline_time': baseline_time,
    'results': all_results,
    'analysis': {
        'best_threads': best_result['n_threads'],
        'best_time': best_result['training_time'],
        'max_speedup': baseline_time / best_result['training_time']
    }
}

with open('results/timings/phase3_cpu_scaling.json', 'w') as f:
    json.dump(cpu_scaling_output, f, indent=2)
print("✓ Saved CPU scaling results: results/timings/phase3_cpu_scaling.json")

# Save summary CSV for easy visualization
summary_df = pd.DataFrame([
    {
        'threads': r['n_threads'],
        'training_time_s': r['training_time'],
        'speedup': baseline_time / r['training_time'],
        'efficiency_pct': (baseline_time / r['training_time'] / r['n_threads']) * 100,
        'test_mae': r['test_mae'],
        'test_r2': r['test_r2']
    }
    for r in all_results
])
summary_df.to_csv('results/timings/phase3_cpu_scaling_summary.csv', index=False)
print("✓ Saved summary CSV: results/timings/phase3_cpu_scaling_summary.csv")

# ====================
# OVERALL SUMMARY
# ====================
phase3_overall_time = time.time() - phase3_overall_start

print("\n" + "="*80)
print("✅ PHASE 3: CPU MODEL TRAINING COMPLETE")
print("="*80)

print(f"\n⏱️  Overall Phase 3 Time: {phase3_overall_time:.2f}s ({phase3_overall_time/60:.2f} minutes)")

print(f"\n📊 Time Breakdown:")
print(f"  Data loading:          {load_time:8.2f}s ({load_time/phase3_overall_time*100:5.1f}%)")
print(f"  Data split:            {split_time:8.2f}s ({split_time/phase3_overall_time*100:5.1f}%)")
print(f"  Categorical encoding:  {prep_time:8.2f}s ({prep_time/phase3_overall_time*100:5.1f}%)")
print(f"  All training runs:     {sum(r['training_time'] for r in all_results):8.2f}s ({sum(r['training_time'] for r in all_results)/phase3_overall_time*100:5.1f}%)")

print(f"\n🎯 Key Results:")
print(f"  • Tested thread counts: {thread_counts}")
print(f"  • Baseline (1 thread): {baseline_time:.2f}s")
print(f"  • Best ({best_result['n_threads']} threads): {best_result['training_time']:.2f}s")
print(f"  • Maximum speedup: {baseline_time/best_result['training_time']:.2f}x")
print(f"  • Model accuracy: R²={best_result['test_r2']:.4f}")

print("\n🚀 Next Steps:")
print("  • Phase 3C: GPU XGBoost training (1/2/4 GPUs)")
print("  • Expected GPU speedup: 3-10x over CPU baseline")
print("  • Phase 4: Hyperparameter optimization")

# Cleanup
print("\n🧹 Cleaning up memory...")
del df, train_df, val_df, test_df
del X_train, X_val, X_test, y_train, y_val, y_test
gc.collect()
print("✓ Memory cleared")

print("\n" + "="*80)
print("✅ Phase 3 execution complete!")
print("="*80)

In [ ]:
zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz

In [ ]:
import subprocess
import sys
import time

print("="*80)
print("STARTING PHASE 4 ")
print("="*80)
print(f"Start time: {time.strftime('%H:%M:%S')}")
print("="*80 + "\n")

start = time.time()

# Run with real-time output
result = subprocess.run(
    [sys.executable, 'phase4_pytorch_cpu.py'],
    capture_output=False
)

elapsed = time.time() - start

print("\n" + "="*80)
if result.returncode == 0:
    print("PHASE 4 COMPLETED SUCCESSFULLY ")
    print(f"Total time: {elapsed:.2f}s ({elapsed/60:.2f} min)")
else:
    print(f" PHASE 4 FAILED")
    print(f"Exit code: {result.returncode}")
    print("Check output above for error details")
print("="*80)

In [ ]:
!python phase4_cpu_16_32_56.py

In [ ]:
#!/usr/bin/env python3
# PHASE 6: Comprehensive Visualizations (25+ plots)
# NYC Real Estate HPC Project

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from scipy import stats

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("="*80)
print("PHASE 6: CREATING 25+ COMPREHENSIVE VISUALIZATIONS")
print("="*80)

os.makedirs('results/plots', exist_ok=True)

# Load data
print("\nLoading data...")
df = pd.read_csv('data/processed/feature_engineered.csv', low_memory=False)
df.columns = df.columns.str.strip()
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'], errors='coerce')
df['SALE_YEAR'] = df['SALE DATE'].dt.year

print(f"  Rows: {len(df):,}, Columns: {len(df.columns)}")

# ============================================================================
# PART A: DATA EXPLORATION (12 plots)
# ============================================================================

print("\n" + "="*80)
print("PART A: DATA EXPLORATION (12 plots)")
print("="*80)

# A1. Price Distribution (2 views)
print("\n[A1] Price Distribution...")
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['SALE PRICE'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Sale Price ($)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Sale Price Distribution', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

mean_price = df['SALE PRICE'].mean()
median_price = df['SALE PRICE'].median()
axes[0].axvline(mean_price, color='red', linestyle='--', linewidth=2, label=f'Mean: ${mean_price:,.0f}')
axes[0].axvline(median_price, color='green', linestyle='--', linewidth=2, label=f'Median: ${median_price:,.0f}')
axes[0].legend()

axes[1].hist(np.log10(df['SALE PRICE'] + 1), bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Log10(Sale Price)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Sale Price (Log Scale)', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/01_price_distribution.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 01_price_distribution.png")

# A2. Sales Over Time
print("\n[A2] Sales Over Time...")
sales_by_year = df.groupby('SALE_YEAR').size()
avg_price_by_year = df.groupby('SALE_YEAR')['SALE PRICE'].mean()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(sales_by_year.index, sales_by_year.values, 'o-', linewidth=2, markersize=6, color='darkblue')
axes[0].axvline(2021.5, color='red', linestyle='--', alpha=0.7, linewidth=2, label='Train/Val')
axes[0].axvline(2022.5, color='orange', linestyle='--', alpha=0.7, linewidth=2, label='Val/Test')
axes[0].set_xlabel('Year', fontsize=12)
axes[0].set_ylabel('Number of Sales', fontsize=12)
axes[0].set_title('NYC Property Sales Over Time', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(avg_price_by_year.index, avg_price_by_year.values, 'o-', linewidth=2, markersize=6, color='green')
axes[1].axvline(2021.5, color='red', linestyle='--', alpha=0.7, linewidth=2)
axes[1].axvline(2022.5, color='orange', linestyle='--', alpha=0.7, linewidth=2)
axes[1].set_xlabel('Year', fontsize=12)
axes[1].set_ylabel('Average Sale Price ($)', fontsize=12)
axes[1].set_title('Average Price Over Time', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/02_sales_over_time.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 02_sales_over_time.png")

# A3. Borough Analysis
print("\n[A3] Borough Analysis...")
borough_names = {1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'Staten Island'}
borough_counts = df['BOROUGH'].value_counts().sort_index()
borough_avg_price = df.groupby('BOROUGH')['SALE PRICE'].mean().sort_index()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

borough_labels = [borough_names.get(b, str(b)) for b in borough_counts.index]
axes[0].bar(range(len(borough_counts)), borough_counts.values, color='teal', edgecolor='black')
axes[0].set_xticks(range(len(borough_counts)))
axes[0].set_xticklabels(borough_labels, rotation=45, ha='right')
axes[0].set_ylabel('Number of Sales', fontsize=12)
axes[0].set_title('Sales by Borough', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

axes[1].bar(range(len(borough_avg_price)), borough_avg_price.values, color='orange', edgecolor='black')
axes[1].set_xticks(range(len(borough_avg_price)))
axes[1].set_xticklabels(borough_labels, rotation=45, ha='right')
axes[1].set_ylabel('Average Sale Price ($)', fontsize=12)
axes[1].set_title('Average Price by Borough', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/03_borough_analysis.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 03_borough_analysis.png")

# A4. Property Age Distribution
print("\n[A4] Property Age Distribution...")
fig, ax = plt.subplots(figsize=(10, 6))

property_ages = df['PROPERTY_AGE'].dropna()
property_ages = property_ages[property_ages >= 0]  # Remove negatives

ax.hist(property_ages, bins=50, color='mediumpurple', edgecolor='black', alpha=0.7)
ax.axvline(property_ages.median(), color='red', linestyle='--', linewidth=2, label=f'Median: {property_ages.median():.0f} years')
ax.set_xlabel('Property Age (years)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of Property Ages', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/04_property_age_distribution.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 04_property_age_distribution.png")

# A5. Building Square Footage Analysis
print("\n[A5] Square Footage Analysis...")
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

gross_sqft = df['GROSS SQUARE FEET'].dropna()
gross_sqft = gross_sqft[(gross_sqft > 0) & (gross_sqft < 50000)]  # Filter extremes

axes[0].hist(gross_sqft, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Gross Square Feet', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Building Size Distribution', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Price per sqft
price_per_sqft = df['PRICE_PER_SQFT'].dropna()
price_per_sqft = price_per_sqft[(price_per_sqft > 0) & (price_per_sqft < 2000)]

axes[1].hist(price_per_sqft, bins=50, color='darkgreen', edgecolor='black', alpha=0.7)
axes[1].axvline(price_per_sqft.median(), color='red', linestyle='--', linewidth=2, 
               label=f'Median: ${price_per_sqft.median():.0f}/sqft')
axes[1].set_xlabel('Price per Square Foot ($/sqft)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Price per Square Foot Distribution', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/05_square_footage_analysis.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 05_square_footage_analysis.png")

# A6. Seasonal Sales Pattern
print("\n[A6] Seasonal Sales Patterns...")
df['SALE_MONTH'] = df['SALE DATE'].dt.month
monthly_sales = df.groupby('SALE_MONTH').size()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.bar(range(1, 13), monthly_sales.values, color='skyblue', edgecolor='black', linewidth=1.5)
ax.set_xticks(range(1, 13))
ax.set_xticklabels(month_names)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Number of Sales', fontsize=12)
ax.set_title('Seasonal Sales Pattern (All Years)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Highlight peak month
peak_month = monthly_sales.idxmax()
bars[peak_month - 1].set_color('darkblue')
bars[peak_month - 1].set_edgecolor('red')
bars[peak_month - 1].set_linewidth(3)

plt.tight_layout()
plt.savefig('results/plots/06_seasonal_pattern.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 06_seasonal_pattern.png")

# A7. Feature Importance
print("\n[A7] Feature Importance...")
import xgboost as xgb

try:
    model = xgb.XGBRegressor()
    model.load_model('models/xgboost_gpu_1.json')
    
    importance = model.get_booster().get_score(importance_type='gain')
    importance_df = pd.DataFrame([
        {'Feature': k, 'Importance': v} 
        for k, v in importance.items()
    ]).sort_values('Importance', ascending=False).head(20)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    
    bars = ax.barh(range(len(importance_df)), importance_df['Importance'].values, 
                   color='mediumpurple', edgecolor='black', linewidth=1)
    ax.set_yticks(range(len(importance_df)))
    ax.set_yticklabels(importance_df['Feature'].values, fontsize=10)
    ax.set_xlabel('Importance (Gain)', fontsize=12)
    ax.set_title('Top 20 Features by Importance', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    ax.invert_yaxis()
    
    plt.tight_layout()
    plt.savefig('results/plots/07_feature_importance.png', dpi=300, bbox_inches='tight')
    print("  ✓ Saved: 07_feature_importance.png")
except Exception as e:
    print(f"  ⚠️  Skipped feature importance: {e}")

# A8. Data Split Visualization
print("\n[A8] Train/Val/Test Split...")
split_data = {
    'Split': ['Train\n(≤2021)', 'Validation\n(2022)', 'Test\n(2023)'],
    'Count': [1125561, 85412, 99973],
    'Percentage': [85.9, 6.5, 7.6]
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#2E86AB', '#A23B72', '#F18F01']
bars = axes[0].bar(split_data['Split'], split_data['Count'], color=colors, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Number of Samples', fontsize=12)
axes[0].set_title('Train/Validation/Test Split', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

for bar, count, pct in zip(bars, split_data['Count'], split_data['Percentage']):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{count:,}\n({pct:.1f}%)',
                ha='center', va='bottom', fontweight='bold', fontsize=10)

axes[1].pie(split_data['Count'], labels=split_data['Split'], autopct='%1.1f%%',
           colors=colors, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Data Split Proportions', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('results/plots/08_data_split.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 08_data_split.png")

# A9. Residential vs Commercial Units
print("\n[A9] Residential vs Commercial Units...")
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

res_units = df['RESIDENTIAL UNITS'].dropna()
res_units = res_units[res_units < 50]

axes[0].hist(res_units, bins=30, color='lightgreen', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Number of Residential Units', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Residential Units Distribution', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

comm_units = df['COMMERCIAL UNITS'].dropna()
comm_units = comm_units[comm_units < 20]

axes[1].hist(comm_units, bins=20, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Number of Commercial Units', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Commercial Units Distribution', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/09_unit_distribution.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 09_unit_distribution.png")

# A10. Neighborhood Diversity
print("\n[A10] Neighborhood Analysis...")
top_neighborhoods = df['NEIGHBORHOOD'].value_counts().head(10)

fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.barh(range(len(top_neighborhoods)), top_neighborhoods.values, color='teal', edgecolor='black')
ax.set_yticks(range(len(top_neighborhoods)))
ax.set_yticklabels(top_neighborhoods.index, fontsize=10)
ax.set_xlabel('Number of Sales', fontsize=12)
ax.set_title('Top 10 Neighborhoods by Sales Volume', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
ax.invert_yaxis()

for i, v in enumerate(top_neighborhoods.values):
    ax.text(v + 500, i, f'{v:,}', va='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.savefig('results/plots/10_top_neighborhoods.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 10_top_neighborhoods.png")

# A11. ZIP Code Analysis
print("\n[A11] ZIP Code Analysis...")
top_zips = df['ZIP CODE'].value_counts().head(15)

fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.bar(range(len(top_zips)), top_zips.values, color='orange', edgecolor='black', linewidth=1.5)
ax.set_xticks(range(len(top_zips)))
ax.set_xticklabels(top_zips.index, rotation=45, ha='right')
ax.set_ylabel('Number of Sales', fontsize=12)
ax.set_title('Top 15 ZIP Codes by Sales Volume', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/11_top_zipcodes.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 11_top_zipcodes.png")

# A12. Building Class Categories
print("\n[A12] Building Class Categories...")
top_classes = df['BUILDING CLASS CATEGORY'].value_counts().head(10)

fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.barh(range(len(top_classes)), top_classes.values, color='purple', edgecolor='black')
ax.set_yticks(range(len(top_classes)))
ax.set_yticklabels(top_classes.index, fontsize=9)
ax.set_xlabel('Number of Sales', fontsize=12)
ax.set_title('Top 10 Building Class Categories', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
ax.invert_yaxis()

plt.tight_layout()
plt.savefig('results/plots/12_building_classes.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 12_building_classes.png")

# ============================================================================
# PART B: PREPROCESSING PERFORMANCE (1 plot)
# ============================================================================

print("\n" + "="*80)
print("PART B: PREPROCESSING PERFORMANCE")
print("="*80)

# B1. Phase 1 Preprocessing Comparison
print("\n[B1] Phase 1 Preprocessing...")
phase1_data = {
    'Method': ['Serial\nPandas', 'Dask\n(threads)', 'joblib\n(processes)'],
    'Time': [25.05, 107.52, 23.36]
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['green', 'red', 'orange']
bars = axes[0].bar(phase1_data['Method'], phase1_data['Time'], color=colors, 
                  edgecolor='black', linewidth=2)
axes[0].axhline(y=25.05, color='green', linestyle='--', alpha=0.6, linewidth=2, label='Serial Baseline')
axes[0].set_ylabel('Time (seconds)', fontsize=12)
axes[0].set_title('Phase 1: Preprocessing Method Comparison', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

for bar in bars:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + 3,
                f'{height:.1f}s', ha='center', va='bottom', fontweight='bold', fontsize=11)

speedups = [25.05 / t for t in phase1_data['Time']]
bars = axes[1].bar(phase1_data['Method'], speedups, color=colors, edgecolor='black', linewidth=2)
axes[1].axhline(y=1.0, color='black', linestyle='--', alpha=0.6, linewidth=2)
axes[1].set_ylabel('Speedup vs Serial', fontsize=12)
axes[1].set_title('Speedup Analysis', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

for bar in bars:
    height = bar.get_height()
    label = f'{height:.2f}x' if height >= 1 else f'{height:.2f}x\n(slower)'
    axes[1].text(bar.get_x() + bar.get_width()/2., max(height, 0.1) + 0.05,
                label, ha='center', va='bottom', fontweight='bold', fontsize=11)

axes[1].text(1, 0.35, '**Key Finding:**\nSerial wins for\nI/O-bound tasks!',
            ha='center', fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.9, edgecolor='black', linewidth=2))

plt.tight_layout()
plt.savefig('results/plots/13_phase1_preprocessing.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 13_phase1_preprocessing.png")

# ============================================================================
# PART C: XGBOOST PERFORMANCE (6 plots)
# ============================================================================

print("\n" + "="*80)
print("PART C: XGBOOST PERFORMANCE (6 plots)")
print("="*80)

# C1. XGBoost CPU Scaling - Training Time
print("\n[C1] XGBoost CPU - Training Time...")
xgb_cpu_data = {
    'Threads': [1, 4, 8, 16, 32, 56],
    'Time': [12.85, 5.02, 3.51, 2.84, 3.01, 2.81]
}

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(xgb_cpu_data['Threads'], xgb_cpu_data['Time'], 'o-', 
       linewidth=3, markersize=12, color='#2E86AB')
ax.set_xlabel('Number of Threads', fontsize=13, fontweight='bold')
ax.set_ylabel('Training Time (seconds)', fontsize=13, fontweight='bold')
ax.set_title('XGBoost CPU: Training Time vs Threads', fontsize=15, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xscale('log', base=2)

for x, y in zip(xgb_cpu_data['Threads'], xgb_cpu_data['Time']):
    ax.text(x, y + 0.3, f'{y:.2f}s', ha='center', va='bottom', fontweight='bold', fontsize=10)

ax.text(8, 10, 'Diminishing returns\nafter 16 threads',
       ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.savefig('results/plots/14_xgb_cpu_time.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 14_xgb_cpu_time.png")

# C2. XGBoost CPU - Speedup
print("\n[C2] XGBoost CPU - Speedup...")
xgb_cpu_speedup = [1.00, 2.56, 3.66, 4.53, 4.27, 4.57]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(xgb_cpu_data['Threads'], xgb_cpu_speedup, 'o-', 
       linewidth=3, markersize=12, color='green', label='Actual', zorder=5)
ax.plot(xgb_cpu_data['Threads'], xgb_cpu_data['Threads'], '--', 
       linewidth=2, alpha=0.6, color='red', label='Ideal (Linear)')

ax.set_xlabel('Number of Threads', fontsize=13, fontweight='bold')
ax.set_ylabel('Speedup', fontsize=13, fontweight='bold')
ax.set_title('XGBoost CPU: Speedup Analysis', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_xscale('log', base=2)
ax.set_yscale('log', base=2)

plt.tight_layout()
plt.savefig('results/plots/15_xgb_cpu_speedup.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 15_xgb_cpu_speedup.png")

# C3. XGBoost CPU - Efficiency
print("\n[C3] XGBoost CPU - Efficiency...")
xgb_cpu_eff = [100.0, 64.0, 45.7, 28.3, 13.4, 8.2]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(xgb_cpu_data['Threads'], xgb_cpu_eff, 'o-', 
       linewidth=3, markersize=12, color='orange')
ax.axhline(y=100, color='red', linestyle='--', alpha=0.6, linewidth=2, label='100% Efficiency')
ax.set_xlabel('Number of Threads', fontsize=13, fontweight='bold')
ax.set_ylabel('Parallel Efficiency (%)', fontsize=13, fontweight='bold')
ax.set_title('XGBoost CPU: Efficiency Degradation', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_xscale('log', base=2)

ax.text(8, 20, 'Efficiency drops\nwith more threads\n(Amdahl\'s Law)',
       ha='center', fontsize=11, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9, edgecolor='black', linewidth=2))

plt.tight_layout()
plt.savefig('results/plots/16_xgb_cpu_efficiency.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 16_xgb_cpu_efficiency.png")

# C4. XGBoost GPU Scaling - Time
print("\n[C4] XGBoost GPU - Training Time...")
xgb_gpu_data = {
    'GPUs': [1, 2, 4],
    'Time': [9.87, 10.09, 9.23]
}

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(range(len(xgb_gpu_data['GPUs'])), xgb_gpu_data['Time'], 
             color=['green', 'darkgreen', 'olive'], edgecolor='black', linewidth=2, width=0.6)
ax.set_xticks(range(len(xgb_gpu_data['GPUs'])))
ax.set_xticklabels(xgb_gpu_data['GPUs'])
ax.set_xlabel('Number of GPUs', fontsize=13, fontweight='bold')
ax.set_ylabel('Training Time (seconds)', fontsize=13, fontweight='bold')
ax.set_title('XGBoost GPU: Training Time Scaling', fontsize=15, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

for i, (bar, v) in enumerate(zip(bars, xgb_gpu_data['Time'])):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.2,
           f'{v:.2f}s', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.savefig('results/plots/17_xgb_gpu_time.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 17_xgb_gpu_time.png")

# C5. XGBoost GPU - Speedup
print("\n[C5] XGBoost GPU - Speedup...")
xgb_gpu_speedup = [1.00, 0.98, 1.07]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(xgb_gpu_data['GPUs'], xgb_gpu_speedup, 'o-', 
       linewidth=3, markersize=12, color='darkgreen', label='Actual')
ax.plot(xgb_gpu_data['GPUs'], [1, 2, 4], '--', 
       linewidth=2, alpha=0.6, color='red', label='Ideal')

ax.set_xlabel('Number of GPUs', fontsize=13, fontweight='bold')
ax.set_ylabel('Speedup', fontsize=13, fontweight='bold')
ax.set_title('XGBoost GPU: Speedup vs Ideal', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

ax.text(2.5, 1.8, 'Limited scaling:\nModel too small\nfor multi-GPU benefit',
       ha='center', fontsize=11, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9, edgecolor='black', linewidth=2))

plt.tight_layout()
plt.savefig('results/plots/18_xgb_gpu_speedup.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 18_xgb_gpu_speedup.png")

# C6. XGBoost GPU - Efficiency
print("\n[C6] XGBoost GPU - Efficiency...")
xgb_gpu_eff = [100.0, 49.0, 26.8]

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(range(len(xgb_gpu_data['GPUs'])), xgb_gpu_eff, 
             color=['purple', 'violet', 'plum'], edgecolor='black', linewidth=2, width=0.6)
ax.axhline(y=100, color='red', linestyle='--', alpha=0.6, linewidth=2, label='100% Efficiency')
ax.set_xticks(range(len(xgb_gpu_data['GPUs'])))
ax.set_xticklabels(xgb_gpu_data['GPUs'])
ax.set_xlabel('Number of GPUs', fontsize=13, fontweight='bold')
ax.set_ylabel('Parallel Efficiency (%)', fontsize=13, fontweight='bold')
ax.set_title('XGBoost GPU: Efficiency Analysis', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(axis='y', alpha=0.3)

for i, (bar, v) in enumerate(zip(bars, xgb_gpu_eff)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 2,
           f'{v:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.savefig('results/plots/19_xgb_gpu_efficiency.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 19_xgb_gpu_efficiency.png")

# ============================================================================
# PART D: PYTORCH CPU PERFORMANCE (3 plots)
# ============================================================================

print("\n" + "="*80)
print("PART D: PYTORCH CPU PERFORMANCE (3 plots)")
print("="*80)

pytorch_cpu_data = {
    'Processes': [4, 16, 32, 56],
    'DDP_Time': [2803.2, 807.8, 910.9, 1060.1],
    'FSDP2_Time': [2638.4, 1226.6, 1450.5, 1707.0]
}

# D1. PyTorch CPU - Time Comparison
print("\n[D1] PyTorch CPU - Time Comparison...")
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(pytorch_cpu_data['Processes']))
width = 0.35

bars1 = ax.bar(x - width/2, pytorch_cpu_data['DDP_Time'], width, label='DDP', 
              color='steelblue', edgecolor='black', linewidth=1.5)
bars2 = ax.bar(x + width/2, pytorch_cpu_data['FSDP2_Time'], width, label='FSDP2', 
              color='coral', edgecolor='black', linewidth=1.5)

ax.set_xlabel('Number of Processes', fontsize=13, fontweight='bold')
ax.set_ylabel('Training Time (seconds)', fontsize=13, fontweight='bold')
ax.set_title('PyTorch CPU: DDP vs FSDP2 (20 epochs)', fontsize=15, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(pytorch_cpu_data['Processes'])
ax.legend(fontsize=12)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/20_pytorch_cpu_time.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 20_pytorch_cpu_time.png")

# D2. PyTorch CPU - DDP Scaling
print("\n[D2] PyTorch CPU - DDP Scaling...")
ddp_baseline = pytorch_cpu_data['DDP_Time'][0]
ddp_speedups = [ddp_baseline / t for t in pytorch_cpu_data['DDP_Time']]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(pytorch_cpu_data['Processes'], ddp_speedups, 'o-', 
       linewidth=3, markersize=12, color='steelblue', label='DDP Actual')
ax.plot(pytorch_cpu_data['Processes'], [1, 4, 8, 14], '--', 
       linewidth=2, alpha=0.6, color='red', label='Ideal')

ax.set_xlabel('Number of Processes', fontsize=13, fontweight='bold')
ax.set_ylabel('Speedup (vs 4 processes)', fontsize=13, fontweight='bold')
ax.set_title('PyTorch CPU DDP: Scaling Analysis', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

ax.text(30, 2.5, 'Good scaling:\n3.47x at 16 processes',
       ha='center', fontsize=11, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.9, edgecolor='black', linewidth=2))

plt.tight_layout()
plt.savefig('results/plots/21_pytorch_cpu_ddp_scaling.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 21_pytorch_cpu_ddp_scaling.png")

# D3. PyTorch CPU - FSDP2 Scaling
print("\n[D3] PyTorch CPU - FSDP2 Scaling...")
fsdp2_baseline = pytorch_cpu_data['FSDP2_Time'][0]
fsdp2_speedups = [fsdp2_baseline / t for t in pytorch_cpu_data['FSDP2_Time']]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(pytorch_cpu_data['Processes'], fsdp2_speedups, 's-', 
       linewidth=3, markersize=12, color='coral', label='FSDP2 Actual')
ax.plot(pytorch_cpu_data['Processes'], [1, 4, 8, 14], '--', 
       linewidth=2, alpha=0.6, color='red', label='Ideal')

ax.set_xlabel('Number of Processes', fontsize=13, fontweight='bold')
ax.set_ylabel('Speedup (vs 4 processes)', fontsize=13, fontweight='bold')
ax.set_title('PyTorch CPU FSDP2: Scaling Analysis', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

ax.text(30, 1.2, 'Limited scaling:\nParameter sharding\noverhead on CPU',
       ha='center', fontsize=11, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.9, edgecolor='black', linewidth=2))

plt.tight_layout()
plt.savefig('results/plots/22_pytorch_cpu_fsdp2_scaling.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 22_pytorch_cpu_fsdp2_scaling.png")

# ============================================================================
# PART E: PYTORCH GPU PERFORMANCE (4 plots)
# ============================================================================

print("\n" + "="*80)
print("PART E: PYTORCH GPU PERFORMANCE (4 plots)")
print("="*80)

pytorch_gpu_data = {
    'GPUs': [1, 2, 4],
    'DDP_Time': [111.7, 78.3, 53.0],
    'FSDP2_Time': [111.7, 106.8, 68.7]
}

# E1. PyTorch GPU - Time Comparison
print("\n[E1] PyTorch GPU - Time Comparison...")
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(pytorch_gpu_data['GPUs']))
width = 0.35

bars1 = ax.bar(x - width/2, pytorch_gpu_data['DDP_Time'], width, label='DDP', 
              color='#2E86AB', edgecolor='black', linewidth=1.5)
bars2 = ax.bar(x + width/2, pytorch_gpu_data['FSDP2_Time'], width, label='FSDP2', 
              color='#A23B72', edgecolor='black', linewidth=1.5)

ax.set_xlabel('Number of GPUs', fontsize=13, fontweight='bold')
ax.set_ylabel('Training Time (seconds)', fontsize=13, fontweight='bold')
ax.set_title('PyTorch GPU: DDP vs FSDP2 (10 epochs)', fontsize=15, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(pytorch_gpu_data['GPUs'])
ax.legend(fontsize=12)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/23_pytorch_gpu_time.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 23_pytorch_gpu_time.png")

# E2. PyTorch GPU - DDP Scaling
print("\n[E2] PyTorch GPU DDP Scaling...")
ddp_gpu_speedup = [1.00, 1.43, 2.11]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(pytorch_gpu_data['GPUs'], ddp_gpu_speedup, 'o-', 
       linewidth=3, markersize=12, color='#2E86AB', label='DDP Actual')
ax.plot(pytorch_gpu_data['GPUs'], [1, 2, 4], '--', 
       linewidth=2, alpha=0.6, color='red', label='Ideal')

ax.set_xlabel('Number of GPUs', fontsize=13, fontweight='bold')
ax.set_ylabel('Speedup', fontsize=13, fontweight='bold')
ax.set_title('PyTorch GPU DDP: Speedup Analysis', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

ax.text(2.5, 2.5, 'Good scaling:\n2.11x at 4 GPUs\n(52.7% efficiency)',
       ha='center', fontsize=11, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.9, edgecolor='black', linewidth=2))

plt.tight_layout()
plt.savefig('results/plots/24_pytorch_gpu_ddp_scaling.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 24_pytorch_gpu_ddp_scaling.png")

# E3. PyTorch GPU - FSDP2 Scaling
print("\n[E3] PyTorch GPU FSDP2 Scaling...")
fsdp2_gpu_speedup = [1.00, 1.05, 1.62]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(pytorch_gpu_data['GPUs'], fsdp2_gpu_speedup, 's-', 
       linewidth=3, markersize=12, color='#A23B72', label='FSDP2 Actual')
ax.plot(pytorch_gpu_data['GPUs'], [1, 2, 4], '--', 
       linewidth=2, alpha=0.6, color='red', label='Ideal')

ax.set_xlabel('Number of GPUs', fontsize=13, fontweight='bold')
ax.set_ylabel('Speedup', fontsize=13, fontweight='bold')
ax.set_title('PyTorch GPU FSDP2: Speedup Analysis', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

ax.text(2.5, 2.2, 'Moderate scaling:\n1.62x at 4 GPUs\n(40.6% efficiency)',
       ha='center', fontsize=11, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9, edgecolor='black', linewidth=2))

plt.tight_layout()
plt.savefig('results/plots/25_pytorch_gpu_fsdp2_scaling.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 25_pytorch_gpu_fsdp2_scaling.png")

# E4. PyTorch GPU - Efficiency Comparison
print("\n[E4] PyTorch GPU - Efficiency Comparison...")
ddp_eff = [100.0, 71.3, 52.7]
fsdp2_eff = [100.0, 52.3, 40.6]

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(pytorch_gpu_data['GPUs']))
width = 0.35

bars1 = ax.bar(x - width/2, ddp_eff, width, label='DDP', 
              color='teal', edgecolor='black', linewidth=1.5)
bars2 = ax.bar(x + width/2, fsdp2_eff, width, label='FSDP2', 
              color='orange', edgecolor='black', linewidth=1.5)

ax.axhline(y=100, color='red', linestyle='--', alpha=0.6, linewidth=2)
ax.set_xlabel('Number of GPUs', fontsize=13, fontweight='bold')
ax.set_ylabel('Parallel Efficiency (%)', fontsize=13, fontweight='bold')
ax.set_title('PyTorch GPU: DDP vs FSDP2 Efficiency', fontsize=15, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(pytorch_gpu_data['GPUs'])
ax.legend(fontsize=12)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/26_pytorch_gpu_efficiency.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 26_pytorch_gpu_efficiency.png")

# ============================================================================
# PART F: ADVANCED ANALYSIS (2 plots)
# ============================================================================

print("\n" + "="*80)
print("PART F: ADVANCED ANALYSIS (2 plots)")
print("="*80)

# F1. Amdahl's Law Analysis
print("\n[F1] Amdahl's Law...")

def amdahl(p, n):
    return 1 / ((1 - p) + p/n)

threads = np.array([1, 4, 8, 16, 32, 56])
measured_speedup = np.array([1.00, 2.56, 3.66, 4.53, 4.27, 4.57])

fig, ax = plt.subplots(figsize=(12, 7))

ax.plot(threads, measured_speedup, 'o-', linewidth=3, markersize=12, 
       label='Measured (XGBoost CPU)', color='blue', zorder=5)

n_range = np.logspace(0, 2.5, 200)

for p, color, style in [(0.90, 'gray', ':'), (0.93, 'purple', '-.'), 
                        (0.95, 'green', '--'), (0.99, 'orange', '--')]:
    speedups = [amdahl(p, n) for n in n_range]
    ax.plot(n_range, speedups, style, alpha=0.7, linewidth=2, 
           label=f'{int(p*100)}% parallelizable', color=color)

ax.plot(n_range, n_range, ':', color='red', alpha=0.6, linewidth=2.5, label='Ideal (100%)')

ax.set_xlabel('Number of Processors', fontsize=13, fontweight='bold')
ax.set_ylabel('Speedup', fontsize=13, fontweight='bold')
ax.set_title("Amdahl's Law: Theoretical Speedup Limits", fontsize=15, fontweight='bold')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3, which='both')

ax.annotate('Measured data:\n~93% parallelizable\n7% sequential bottleneck',
            xy=(56, 4.57), xytext=(25, 12),
            arrowprops=dict(arrowstyle='->', color='black', lw=2),
            fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.9, edgecolor='black', linewidth=2))

ax.text(0.98, 0.05, 'Maximum theoretical speedup: ~14x\n(limited by 7% sequential fraction)',
       transform=ax.transAxes, ha='right', va='bottom', fontsize=10,
       bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))

plt.tight_layout()
plt.savefig('results/plots/27_amdahl_law_analysis.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 27_amdahl_law_analysis.png")

# F2. HPO GPU Results
print("\n[F2] Hyperparameter Optimization Results...")
hpo_gpu_time = 2047.7
best_params = {'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 5, 'n_estimators': 100}
best_mae = 25047.88
best_r2 = 0.9960

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].text(0.5, 0.7, 'XGBoost GPU\nHyperparameter Optimization', 
            ha='center', va='center', fontsize=18, fontweight='bold',
            transform=axes[0].transAxes)
axes[0].text(0.5, 0.5, f'Time: {hpo_gpu_time:.0f}s ({hpo_gpu_time/60:.1f} min)', 
            ha='center', va='center', fontsize=15,
            transform=axes[0].transAxes,
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
axes[0].text(0.5, 0.3, '81 combinations × 5 folds\n= 405 model fits', 
            ha='center', va='center', fontsize=13,
            transform=axes[0].transAxes)
axes[0].axis('off')

param_text = '\n'.join([f'{k}: {v}' for k, v in best_params.items()])
axes[1].text(0.5, 0.70, 'Best Parameters Found:', 
            ha='center', va='center', fontsize=15, fontweight='bold',
            transform=axes[1].transAxes)
axes[1].text(0.5, 0.48, param_text, 
            ha='center', va='center', fontsize=13, family='monospace',
            transform=axes[1].transAxes,
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8, edgecolor='black', linewidth=2))
axes[1].text(0.5, 0.22, f'Test MAE: ${best_mae:,.0f}', 
            ha='center', va='center', fontsize=14, fontweight='bold',
            transform=axes[1].transAxes)
axes[1].text(0.5, 0.10, f'Test R²: {best_r2:.4f}', 
            ha='center', va='center', fontsize=14, fontweight='bold',
            transform=axes[1].transAxes)
axes[1].axis('off')

plt.suptitle('GridSearchCV Results on GPU', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig('results/plots/28_hpo_results.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 28_hpo_results.png")

# ============================================================================
# PART G: SUMMARY & INSIGHTS (2 plots)
# ============================================================================

print("\n" + "="*80)
print("PART G: SUMMARY VISUALIZATIONS (2 plots)")
print("="*80)

# G1. Complete Performance Table
print("\n[G1] Performance Summary Table...")
summary_data = {
    'Phase': [
        'Preprocessing', 'Preprocessing', 'Preprocessing',
        'XGBoost CPU', 'XGBoost CPU',
        'XGBoost GPU', 'XGBoost GPU', 'XGBoost GPU',
        'PyTorch CPU DDP', 'PyTorch CPU DDP', 'PyTorch CPU DDP',
        'PyTorch CPU FSDP2', 'PyTorch CPU FSDP2', 'PyTorch CPU FSDP2',
        'PyTorch GPU DDP', 'PyTorch GPU DDP', 'PyTorch GPU DDP',
        'PyTorch GPU FSDP2', 'PyTorch GPU FSDP2', 'PyTorch GPU FSDP2',
        'HPO GPU'
    ],
    'Configuration': [
        'Serial', 'Dask (16)', 'joblib (16)',
        '1 thread', '56 threads',
        '1 GPU', '2 GPUs', '4 GPUs',
        '4 proc', '16 proc', '56 proc',
        '4 proc', '16 proc', '56 proc',
        '1 GPU', '2 GPUs', '4 GPUs',
        '1 GPU', '2 GPUs', '4 GPUs',
        'GridSearchCV'
    ],
    'Time': [
        '25.1s', '107.5s', '23.4s',
        '12.9s', '2.8s',
        '9.9s', '10.1s', '9.2s',
        '46.7min', '13.5min', '17.7min',
        '44.0min', '20.4min', '28.5min',
        '111.7s', '78.3s', '53.0s',
        '111.7s', '106.8s', '68.7s',
        '34.1min'
    ],
    'Accuracy': [
        '-', '-', '-',
        'R²=0.9941', 'R²=0.9941',
        'R²=0.9948', 'R²=0.9948', 'R²=0.9948',
        'R²=0.9998', '-', '-',
        'R²=0.9998', '-', '-',
        'R²=0.9994', 'R²=0.9992', 'R²=0.9934',
        'R²=0.9994', 'R²=0.9984', 'R²=0.9993',
        'R²=0.9960'
    ]
}

df_summary = pd.DataFrame(summary_data)
df_summary.to_csv('results/complete_performance_summary.csv', index=False)

fig, ax = plt.subplots(figsize=(14, 11))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_summary.values, colLabels=df_summary.columns,
                cellLoc='left', loc='center', colColours=['#4472C4']*4)

table.auto_set_font_size(False)
table.set_fontsize(8)
table.scale(1, 2.2)

for i in range(4):
    table[(0, i)].set_facecolor('#4472C4')
    table[(0, i)].set_text_props(weight='bold', color='white', fontsize=9)

for i in range(1, len(df_summary) + 1):
    for j in range(4):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#F2F2F2')

plt.title('Complete Performance Summary', fontsize=16, fontweight='bold', pad=20)
plt.savefig('results/plots/29_complete_summary_table.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 29_complete_summary_table.png")

# G2. Key Findings Infographic
print("\n[G2] Key Findings Summary...")
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111)
ax.axis('off')

fig.text(0.5, 0.96, 'NYC Real Estate HPC Project - Key Findings', 
        ha='center', fontsize=22, fontweight='bold')

findings = [
    ("Data: 2M → 1.3M Rows", "Cleaned 35% bad data | 18 engineered features | Time-aware split"),
    ("Phase 1: Preprocessing", "Serial (25s) FASTER than parallel | I/O-bound: overhead > gains"),
    ("XGBoost CPU: 1→56 Threads", "12.85s → 2.81s | 4.57x speedup | 8.2% efficiency (Amdahl's Law)"),
    ("XGBoost GPU: 1→4 GPUs", "9.87s → 9.23s | 1.07x speedup | Small model: limited scaling"),
    ("PyTorch CPU DDP: 4→56 Processes", "2803s → 1060s | 3.47x speedup at 16 proc | Degrades at 56"),
    ("PyTorch CPU FSDP2: 4→56 Processes", "2638s → 1707s | Limited scaling | Sharding overhead on CPU"),
    ("PyTorch GPU DDP: 1→4 GPUs", "111.7s → 53.0s | 2.11x speedup | 52.7% efficiency"),
    ("PyTorch GPU FSDP2: 1→4 GPUs", "111.7s → 68.7s | 1.62x speedup | Memory-efficient but slower"),
    ("HPO: GPU GridSearchCV", "34.1 min for 405 fits | Found: depth=9, lr=0.1 | R²=0.9960"),
    ("Overall Accuracy", "XGBoost: R²=0.9941-0.9948 | PyTorch: R²=0.9934-0.9998 | Excellent!")
]

y_positions = np.linspace(0.87, 0.12, len(findings))
colors_cycle = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral', 'lavender']

for i, (title, text) in enumerate(findings):
    color = colors_cycle[i % len(colors_cycle)]
    
    ax.add_patch(plt.Rectangle((0.05, y_positions[i] - 0.065), 0.9, 0.075,
                               facecolor=color, edgecolor='black', linewidth=2, alpha=0.85,
                               transform=fig.transFigure))
    
    fig.text(0.08, y_positions[i] + 0.005, title,
            fontsize=12, fontweight='bold', transform=fig.transFigure)
    
    fig.text(0.08, y_positions[i] - 0.032, text,
            fontsize=10, transform=fig.transFigure)

plt.savefig('results/plots/30_key_findings.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: 30_key_findings.png")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*80)
print("✓✓✓ ALL 30 VISUALIZATIONS COMPLETE ✓✓✓")
print("="*80)
print("\n📊 Created 30 publication-quality plots:")
print("\n  PART A: Data Exploration (7 plots)")
print("    01. Price distribution (2 views)")
print("    02. Sales over time")
print("    03. Borough analysis")
print("    04. Property age distribution")
print("    05. Square footage analysis")
print("    06. Seasonal sales pattern")
print("    07. Feature importance (top 20)")
print("    08. Train/val/test split")
print("    09. Residential/commercial units")
print("    10. Top neighborhoods")
print("    11. Top ZIP codes")
print("    12. Building class categories")
print("\n  PART B: Preprocessing (1 plot)")
print("    13. Phase 1 comparison")
print("\n  PART C: XGBoost (6 plots)")
print("    14. CPU - Training time")
print("    15. CPU - Speedup")
print("    16. CPU - Efficiency")
print("    17. GPU - Training time")
print("    18. GPU - Speedup")
print("    19. GPU - Efficiency")
print("\n  PART D: PyTorch CPU (3 plots)")
print("    20. DDP vs FSDP2 time")
print("    21. DDP scaling")
print("    22. FSDP2 scaling")
print("\n  PART E: PyTorch GPU (4 plots)")
print("    23. DDP vs FSDP2 time")
print("    24. DDP scaling")
print("    25. FSDP2 scaling")
print("    26. Efficiency comparison")
print("\n  PART F: Advanced Analysis (2 plots)")
print("    27. Amdahl's Law")
print("    28. HPO results")
print("\n  PART G: Summary (2 plots)")
print("    29. Complete performance table")
print("    30. Key findings infographic")
print("\n📁 All saved to: results/plots/")
print("📄 CSV summary: results/complete_performance_summary.csv")
print("="*80)
print("\n✨ Ready for final report!")
print("="*80)